In [1]:
using Random
using LinearAlgebra
using SparseArrays
using PyCall
using Statistics

In [2]:
L = 2;
Number_Of_Noise = 4*L^2-6*L+13-3;
Random.seed!(2022)
NOISE = 2*rand(Float64,Number_Of_Noise).-1;

In [3]:
#Rx(theta) = exp(-1im*(theta*[-1 1;1 -1]/2));
#Rx(theta) = [cos(theta/2) -1im*sin(theta/2) ; -1im*sin(theta/2)  cos(theta/2)];
Rx(theta) = exp(-1im*theta*[1 1;1 1]/2);

In [4]:
int(x) = floor(Int,x)

int (generic function with 1 method)

In [5]:
#Rx(theta) = exp(-1im*theta*[1 1;1 1]/2);
#Rx(theta) = [cos(theta/2) -1im*sin(theta/2) ; -1im*sin(theta/2)  cos(theta/2)];#

#round.(-exp(-1im*pi*([1 1;1 1]/2)); digits = 3)

Ry(theta) = [cos(theta/2) -sin(theta/2) ; sin(theta/2) cos(theta/2)];

Pauli_X = [0 1;1 0];
Pauli_Y = [1 -1im;1im 0];
Pauli_Z = [1 0;0 -1];
X = [0 1;1 0];
I2 = [1 0; 0 1];
Z = [1 0;0 -1];
H = (1/sqrt(2))*[1 1;1 -1]
Hadamard(noise) = exp(-1im*(pi/2+noise)*(I2-H)) #Ry(pi/2+noise)*Pauli_Z;
#CX(noise) = exp(-1im*(pi/2+noise)*(X-I2));
CX(noise) = exp(-1im*(pi+noise)*[1 1;1 1]/2);

In [6]:
"""

Following function takes a 2x2 matrix (Gate) and qubit position (Qubit) and
returns the resultant matrix.

For example, the matrix for the gate U acting on the 3-rd qubit for N=5
qubit system is given by   I (x) I (x) U (x) I (x) I; where (x) is the
tensor product.

"""

function Matrix_Gate(Gate, Qubit) # Previously known as multi qubit gate.
    
    ## The case Qubit=1 is treated differently because we need to
    # initialize the matrix as U before starting the kronecker product.
    
    if Qubit == 1
        
        M = sparse(Gate)
        for i=2:L
            M = kron(M, sparse([1 0;0 1]))
        end
        
    else
        
        M = sparse([1 0;0 1])
        for i=2:L
            if i == Qubit
                M = kron(M, Gate)
            else
                M = kron(M, sparse([1 0;0 1]))
            end
        end
    end
    
    return M
end;

In [7]:
Identity(dimension) = 1* Matrix(I, dimension, dimension);

In [8]:
"""

The following function returns a controlled U gate matrix.

Input  : c (integer), t(integer), U (unitary operator).
Output : Matrix of the multicontrolled U gate with control qubit c and target qubit t.

"""

function CU(U,c,t)
    
    I2 = sparse([1 0;0 1])
    Z = sparse([1 0;0 -1])

    PI_0 = (I2+Z)/2
    PI_1 = (I2-Z)/2
     
    #function Rx(Noise)
        #A = cos((pi+Noise)/2)
        #B = -1im*sin((pi+Noise)/2)
        #return 1im*[A B;B A]
    #end
    
    Matrices = Dict("I" => I2,"PI_0" => PI_0,"U" => U, "PI_1" => PI_1)
    
    p0 = fill("I", L)
    p1 = fill("I", L)
    
    p0[c] = "PI_0"
    p1[c] = "PI_1"
    p1[t] = "U"

    
    PI_0_matrix = Matrices[p0[1]]
    for i = 2:L
        PI_0_matrix = kron(PI_0_matrix,Matrices[p0[i]])
    end        
        
    PI_1_matrix = Matrices[p1[1]]   
    for i = 2:L
        PI_1_matrix = kron(PI_1_matrix,Matrices[p1[i]])        
    end
           
    #return p0,p1
    return PI_0_matrix + PI_1_matrix     
end;

In [9]:
"""

The following returns a multicontrolled U gate matrix.

Input  : c (list), t(integer), U (unitary operator).
Output : Matrix of the multicontrolled U gate with control qubits c and target qubit t.

"""

function MCU(c,t,U)
    
    p0 = fill("I", L)
    p1 = fill("I", L)

    
    if typeof(c) == Int64
        p0[c] = "PI_1"
        p1[t] = "PI_1"
        
    else
        for i in c
            p0[i] = "PI_1"
            p1[i] = "PI_1"
        end
    end
    
    p0[t] = "I"
    p1[t] = "U"

    
    I = sparse([1 0;0 1])
    Z = sparse([1 0;0 -1])
    X = sparse([0 1;1 0])
    PI_0 = (I+Z)/2
    PI_1 = (I-Z)/2
     
    Matrices = Dict("I" => I,"PI_0" => PI_0,"U" => U, "PI_1" => PI_1)
    
    PI_0_matrix = Matrices[p0[1]]
    for i = 2:L
        PI_0_matrix = kron(PI_0_matrix,Matrices[p0[i]])
    end        
        
    PI_1_matrix = Matrices[p1[1]]   
    for i = 2:L
        PI_1_matrix = kron(PI_1_matrix,Matrices[p1[i]])        
    end
             
    # The identity in the following line needs to be replaced.
    return Identity(2^L) - PI_0_matrix + PI_1_matrix     
end;

In [10]:
using PyCall
py"""
import numpy
import numpy.linalg
def adjoint(psi):
    return psi.conjugate().transpose()
def psi_to_rho(psi):
    return numpy.outer(psi,psi.conjugate())
def exp_val(psi, op):
    return numpy.real(numpy.dot(adjoint(psi),op.dot(psi)))
def norm_sq(psi):
    return numpy.real(numpy.dot(adjoint(psi),psi))
def normalize(psi,tol=1e-9):
    ns=norm_sq(psi)**0.5
    if ns < tol:
        raise ValueError
    return psi/ns
def is_herm(M,tol=1e-9):
    if M.shape[0]!=M.shape[1]:
        return False
    diff=M-adjoint(M)
    return max(numpy.abs(diff.flatten())) < tol
def is_unitary(M,tol=1e-9):
    if M.shape[0]!=M.shape[1]:
        return False
    diff=M.dot(adjoint(M))-numpy.identity((M.shape[0]))
    return max(numpy.abs(diff.flatten())) < tol
def eigu(U,tol=1e-9):
    (E_1,V_1)=numpy.linalg.eigh(U+adjoint(U))
    U_1=adjoint(V_1).dot(U).dot(V_1)
    H_1=adjoint(V_1).dot(U+adjoint(U)).dot(V_1)
    non_diag_lst=[]
    j=0
    while j < U_1.shape[0]:
        k=0
        while k < U_1.shape[0]:
            if j!=k and abs(U_1[j,k]) > tol:
                if j not in non_diag_lst:
                    non_diag_lst.append(j)
                if k not in non_diag_lst:
                    non_diag_lst.append(k)
            k+=1
        j+=1
    if len(non_diag_lst) > 0:
        non_diag_lst=numpy.sort(numpy.array(non_diag_lst))
        U_1_cut=U_1[non_diag_lst,:][:,non_diag_lst]
        (E_2_cut,V_2_cut)=numpy.linalg.eigh(1.j*(U_1_cut-adjoint(U_1_cut)))
        V_2=numpy.identity((U.shape[0]),dtype=V_2_cut.dtype)
        for j in range(len(non_diag_lst)):
            V_2[non_diag_lst[j],non_diag_lst]=V_2_cut[j,:]
        V_1=V_1.dot(V_2)
        U_1=adjoint(V_2).dot(U_1).dot(V_2)
    # Sort by phase
    U_1=numpy.diag(U_1)
    inds=numpy.argsort(numpy.imag(numpy.log(U_1)))
    return (U_1[inds],V_1[:,inds]) # = (U_d,V) s.t. U=V*U_d*V^\dagger
"""

### Exact Grover operator

In [11]:
U_0 = [-1 0 0 0; 0 1 0 0; 0 0 1 0;0 0 0 1];
A = ones(2^L,2^L);
U_x = (2/2^L)*A-Identity(2^L);
G_exact = U_x*U_0
#U_x

4×4 Matrix{Float64}:
  0.5   0.5   0.5   0.5
 -0.5  -0.5   0.5   0.5
 -0.5   0.5  -0.5   0.5
 -0.5   0.5   0.5  -0.5

### Decomposed Grover operator

In [12]:
U0 = Matrix_Gate(CX(0.0),1)*Matrix_Gate(H,2)*CU(CX(0.0),1,2)*Matrix_Gate(CX(0.0),1)*Matrix_Gate(H,2);
Ux = Matrix_Gate(H,1)*Matrix_Gate(CX(0.0),1)*CU(CX(0.0),1,2)*Matrix_Gate(CX(0.0),1)*Matrix_Gate(H,1);
G_decomposed = real(round.(-Ux*U0,digits=2))
#real(round.(-Ux,digits=2))

4×4 SparseMatrixCSC{Float64, Int64} with 16 stored entries:
  0.5   0.5   0.5   0.5
 -0.5  -0.5   0.5   0.5
 -0.5   0.5  -0.5   0.5
 -0.5   0.5   0.5  -0.5

### Construction of H_eff

In [13]:
    function kth_term(k)
        
        f_k = Identity(2^L);
        for i = k:length(List_of_U)-1
            f_k = f_k * collect(List_of_U[length(List_of_U)-i+k])
        end 
        
        #= Corresponding H for the kth term. =#
        U_k = List_of_H[k]
        if U_k[1] == "H"
            Noise = U_k[2] # delta*epsilon.
            Qubit = U_k[3] # qubit.
            H_k = Matrix_Gate(I2-H,Qubit) #= H_had = I2-Had. =#
        elseif U_k[1] == "X"
            Noise = U_k[2] # delta*epsilon.
            Qubit = U_k[3] # qubit.
            H_k = Matrix_Gate(X-I2,Qubit) #= H_X = X-I2. =#
 
        else
            Noise = U_k[1]
            Control_Qubit = int(U_k[2])
            Target_Qubit = int(U_k[3])
            #= H = ((I-Z)/2)_c \otimes ((I+X)/2)_t.=#
            Matrices = Dict("I" => I2,"U" => (I2+X)/2, "PI_1" => (I2-Z)/2)
            p1 = fill("I", L)
            p1[Control_Qubit] = "PI_1"
            p1[Target_Qubit] = "U"
            H_k = Matrices[p1[1]]
            for i = 2:L
                H_k = kron(H_k,Matrices[p1[i]])
            end                                 
        end
        return f_k * H_k * (f_k')
end;

In [14]:
kth_term(10)

LoadError: UndefVarError: List_of_U not defined

In [15]:
py"""
f = open('eigenvalues_manual_data'+'.txt', 'w')
def Write_file2(delta, effective, exact):
    f = open('eigenvalues_manual_data'+'.txt', 'a')
    f.write(str(delta) + '\t' + str(effective)+ '\t' + str(exact) +'\n')
"""

In [16]:
Num = 80;
for i = 1:Num
    delta = 0.3*(i/Num)
    EE = MyEigenvalues(delta);
    effective = EE[1]
    exact = EE[2]
    for j = 1:2^L-2
        py"Write_file2"(delta,effective[j],exact[j])
    end
end

LoadError: UndefVarError: MyEigenvalues not defined